# Data Visualization Individual Plots

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import json

## Get datasets

In [ ]:
data_path = '../data/'
pct_data = pd.read_csv(data_path + 'renewables_percent_timeseries.csv')
pct_data = pct_data.sort_values(by='Country')

In [ ]:
#pct_data#['SIEC Code'].unique()

In [ ]:
ts_years = [str(i) for i in range(2004, 2021)]

percent_timeseries = pct_data.loc[pct_data['SIEC Code']=='RA000']\
                              .reset_index(drop=True)
percent_timeseries_tot = pct_data.loc[pct_data['SIEC Code']=='TOTAL']\
                                 .reset_index(drop=True)

for yi in ts_years:
    percent_timeseries[yi] = round((100*percent_timeseries[yi] / percent_timeseries_tot[yi]), 2)
#percent_timeseries

In [ ]:
pct_data.columns

In [ ]:
rs_data = pd.read_csv(data_path + 'renewable_sources_2020.csv')
#rs_data

## Plot Percent Consumption

In [ ]:
df_country_points = pd.read_csv(data_path + 'country_points.csv')

data_geo = dict()

with open(data_path + 'countries.geojson') as json_file:
    data_geo = json.load(json_file)

In [ ]:
for feature in data_geo['features']:
    feature['id'] = feature['properties']['ADMIN']   # or 'ISO-A3'
    
fr = data_geo['features'][77]['geometry']['coordinates'][0][0][0]


In [ ]:
data_scattermap = dict(type='choroplethmapbox', 
                       geojson=data_geo,
                      locations=percent_timeseries['Country'], 
                      z=percent_timeseries['2020'],


                      colorscale='viridis',
                      colorbar=dict(title='Percent Renewables')
                      )


layout_scattermap = dict(mapbox=dict(style='white-bg',
                                     center={"lat": 56.5, "lon": 11},
                                     zoom=2.5,
                                     layers=[dict(source=feature,
                                                  below='traces',
                                                  type='fill',
                                                  
                                                  fill=dict(outlinecolor='gray')
                                                         ) for feature in data_geo['features']]
                                    ),
                         title=dict(text='World Map',
                                    x=.5 # Title relative position according to the xaxis, range (0,1)
                                   ),
                         
                           )

In [ ]:
fig_scattermap = go.Figure(data=data_scattermap, layout=layout_scattermap)


In [ ]:
#fig_scattermap.show()#renderer='browser')

In [ ]:
# An empty graph object
fig_bar = go.Figure()

# Each year defines a new hidden (implies visible=False) trace in our visualization
for year in ts_years:
    fig_bar.add_trace(dict(type='bar',
                     x=percent_timeseries['Country'],
                     y=percent_timeseries[year],
                     name=year,
                     showlegend=False,
                     visible=False
                    )
               )

# First seen trace
fig_bar.data[0].visible = True


# Lets create our slider, one option for each trace
steps = []
for i in range(len(fig_bar.data)):
    step = dict(
        label='Year ' + ts_years[i],
        method="restyle", #there are four methods restyle changes the type of an argument (in this case if visible or not)
        args=["visible", [False] * len(fig_bar.data)], # Changes all to Not visible
    )
    step["args"][1][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

    
sliders = [dict(
    active=2020,
    pad={"t": 100},
    steps=steps
)]


fig_bar.update_layout(dict(title=dict(text='Percentage of consumption using renewables'),
                  yaxis=dict(title='Percentage',
                             range=[0,100]
                            ),
                  sliders=sliders))



In [ ]:
#fig_bar.show(renderer='iframe')